   # Projeto 2 - Ciência do Dados
___

   ## Arthur Pansini; Enzo Fonteyne; Andre Faia.

   Nosso projeto é focado na industria automobilistica, na qual utilizamos das variaveis ano de fabricação, modelo, tipo de câmbio e quilometros rodados para analizar quais dessas caracteristicas estão mais presentes em carros vendidos acima dos 20 mil dolares. Buscamos trazer uma base de dados ampla e com enorme diversidade de informações acerca do tema, para que se aproxime o maximo possível da realidade!

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data = pd.read_csv("train.csv")
data.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [7]:
filtro = data.loc[:,["Price", "Prod. year", "Category", "Manufacturer",  "Model","Gear box type","Mileage"]]

In [8]:
filtro

,Price,Prod. year,Category,Manufacturer,Model,Gear box type,Mileage
0,13328,2010,Jeep,LEXUS,RX 450,Automatic,186005 km
1,16621,2011,Jeep,CHEVROLET,Equinox,Tiptronic,192000 km
2,8467,2006,Hatchback,HONDA,FIT,Variator,200000 km
3,3607,2011,Jeep,FORD,Escape,Automatic,168966 km
4,11726,2014,Hatchback,HONDA,FIT,Automatic,91901 km
...,...,...,...,...,...,...,...
19232,8467,1999,Coupe,MERCEDES-BENZ,CLK 200,Manual,300000 km
19233,15681,2011,Sedan,HYUNDAI,Sonata,Tiptronic,161600 km
19234,26108,2010,Jeep,HYUNDAI,Tucson,Automatic,116365 km
19235,5331,2007,Jeep,CHEVROLET,Captiva,Automatic,51258 km


In [9]:
filtro.isnull().sum()

Price            0
Prod. year       0
Category         0
Manufacturer     0
Model            0
Gear box type    0
Mileage          0
dtype: int64

In [10]:
filtro.tail(10)

,Price,Prod. year,Category,Manufacturer,Model,Gear box type,Mileage
19227,29793,2014,Sedan,MERCEDES-BENZ,E 350,Automatic,219030 km
19228,706,2008,Sedan,MERCEDES-BENZ,E 350,Automatic,122874 km
19229,50,2008,Hatchback,TOYOTA,Prius,Automatic,150000 km
19230,470,2011,Hatchback,TOYOTA,Prius,Automatic,307325 km
19231,5802,2013,Sedan,MERCEDES-BENZ,E 350,Automatic,107800 km
19232,8467,1999,Coupe,MERCEDES-BENZ,CLK 200,Manual,300000 km
19233,15681,2011,Sedan,HYUNDAI,Sonata,Tiptronic,161600 km
19234,26108,2010,Jeep,HYUNDAI,Tucson,Automatic,116365 km
19235,5331,2007,Jeep,CHEVROLET,Captiva,Automatic,51258 km
19236,470,2012,Sedan,HYUNDAI,Sonata,Automatic,186923 km


In [11]:
filtro.drop(filtro.loc[filtro['Price']<=500].index, inplace=True)

In [89]:
filtro.tail(10)

,Price,Prod. year,Category,Manufacturer,Model,Gear box type,Mileage
19224,19130,2014,Sedan,KIA,Optima ex,Automatic,35800 km
19225,8781,2007,Goods wagon,OPEL,Combo,Manual,236000 km
19226,7840,2003,Sedan,NISSAN,Skyline,Tiptronic,220000 km
19227,29793,2014,Sedan,MERCEDES-BENZ,E 350,Automatic,219030 km
19228,706,2008,Sedan,MERCEDES-BENZ,E 350,Automatic,122874 km
19231,5802,2013,Sedan,MERCEDES-BENZ,E 350,Automatic,107800 km
19232,8467,1999,Coupe,MERCEDES-BENZ,CLK 200,Manual,300000 km
19233,15681,2011,Sedan,HYUNDAI,Sonata,Tiptronic,161600 km
19234,26108,2010,Jeep,HYUNDAI,Tucson,Automatic,116365 km
19235,5331,2007,Jeep,CHEVROLET,Captiva,Automatic,51258 km
